# Content-Based Filtering Script

In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [3]:


similarity_list=["cosine","pearson","adjusted","asymmetric"]
vals=[0,10,50,100]

recommender = ItemKNNCBFRecommender(URM_train, ICM_all)
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])
resultDF= pd.DataFrame({"similarity":[],"shrink":[],"topK":[],"MAP":[]})

for similarity in similarity_list:
    
    for shrink in vals:
        
        for topK in vals:
            
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity)
            MAP,_=evaluator_validation.evaluateRecommender(recommender)
            MAP=MAP[10]["MAP"]
            resultDF=resultDF.append({"similarity":[similarity],"shrink":[shrink],"topK":[topK],"MAP":[MAP]},ignore_index=True)
            print(MAP)
            
resultDF.to_csv("CBF_results")

ItemKNNCBFRecommender: URM Detected 88 (1.11 %) cold users.
ItemKNNCBFRecommender: URM Detected 2403 (9.25 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 1893.09 column/sec, elapsed time 0.23 min
EvaluatorHoldout: Processed 5654 ( 100.00% ) in 2.23 sec. Users per second: 2532
0.0002453242822892628
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 1693.57 column/sec, elapsed time 0.26 min
EvaluatorHoldout: Processed 5654 ( 100.00% ) in 2.47 sec. Users per second: 2288
0.018122294539401267
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 1612.34 column/sec, elapsed time 0.27 min
EvaluatorHoldout: Processed 5654 ( 100.00% ) in 2.60 sec. Users per second: 2178
0.02164459318294782
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 1559.57 column/sec, elapsed time 0.28 min
EvaluatorHoldout

In [6]:
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("CBF_results_sorted")



In [7]:
recommender_full= ItemKNNCBFRecommender(URM_all, ICM_all)
recommender_full.fit(topK=100, shrink=10, similarity='cosine')

ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 1530.04 column/sec, elapsed time 0.28 min


In [35]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)
for uid in target_user_list:
    rec_list=map(str,recommender_full.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("CBF_recomm_100_10_cosine.csv")

0
1
2
